In [2]:
import requests as req
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from time import sleep
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.common.exceptions import NoSuchElementException


## input URL

In [2]:
url = 'http://www.ogb.state.ms.us/prodsearch.php'

In [7]:
fields = []
res = req.get(url)
print(res.status_code)
soup = BeautifulSoup(res.text,'lxml')
fields_option = soup.find("select", id='FieldName').find_all('option')
for field_name in fields_option:
    field = field_name.text
    if field != 'No Preference':
        fields.append(field)
            

200


## field data to crawl

In [8]:
def get_page_data(soup):
    '''get table data from passed in soup
    input;soup
    output:dataframe with columns name
    '''
    field_column_name = ['Well Info','Report Period','API (10)','OperatorName','Well Name','Oil (bbls)','Water (bbls)','Gas (mcf)'
                         ,'Gas Tarns','Field Name','Pool Name','Country Name','Well Type','Well Status']
    table_row_id = "example"
     ## find the table data inside the soup
    table_data = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']==table_row_id)
    rows = table_data.findAll(lambda tag: tag.name=='tr'and  tag.has_attr('class'))
    df_row = list()
    ## scrapy data
    for j in range(len(rows)): 
        ## for each datable row
        tds = rows[j].find_all('td')
#             print(tds)
#             print(j)
        df_col = list()
        for k in range(len(tds)):
#                 print(k)
            value = tds[k].text
            ## df define is the problem
            df_col.append(value)
#                 print(k)  
#         print(df_col)
        df_row.append(df_col)
    ## modify df_row to dataframe
    df = pd.DataFrame (df_row,columns=field_column_name)
    
    return df

        

## function drive the explorer

In [11]:
def get_field_page_source(url ='http://www.ogb.state.ms.us/prodsearch.php'):
    ''' function to choose a certain field from option list
        and load the production web
    '''
    table_row_id = "example"
    
    field_data = {}
    field_data_each_page = []
    ##using soup to get fields list
    fields = []
    res = req.get(url)
    print(res.status_code)
    soup = BeautifulSoup(res.text,'lxml')
    fields_option = soup.find("select", id='FieldName').find_all('option')
    for field_name in fields_option:
        field = field_name.text
        if field != 'No Preference':
            fields.append(field)
            
#     print(fields[0:5])
    ##select field name and submit request
    for field in fields[0:40] :
#         if field not in field_first_crawl:
            ## initiate the driver
        try:
            driver = webdriver.Chrome()
            driver.get(url)
            sleep(3)
            select_element = driver.find_element_by_id('FieldName')
            select_object = Select(select_element)
            select_object.select_by_value(field)
            driver.find_element(By.XPATH, '//button[1]').submit()

            ## parse new page_source as a soup
            first_page_soup = BeautifulSoup(driver.page_source,'html.parser')
            ##get the data from current page
            first_page_data = get_page_data(first_page_soup)
            field_data_each_page.append(first_page_data)

            ##browsing all the left pages
            ## get the maximum page number
            pages = first_page_soup.find_all('a',attrs = {'class':'paginate_button'})
    #         print(pages)
            page_list= []
            for page in pages:
                page_num  = page.text
                page_list.append(page_num) 
    #         print(page_list)
            click_next_times = int(page_list[-2])

            ##click the 'next_page ' button
            for i in range(click_next_times):
                driver.find_element(By.CSS_SELECTOR, 'a.next').click()
                next_page_source = driver.page_source
                next_page_soup = BeautifulSoup(next_page_source,'html.parser')
                other_page_data = get_page_data(next_page_soup)
                field_data_each_page.append(other_page_data)

            ## save each field data to field_data dictionary
            field_data[field] = field_data_each_page
            driver.close()
            print(field,'Field data is saved')
        except NoSuchElementException:
            print("Cannot locate option with value: Doak's Creek")
        
    return field_data

## Run the crawling

In [12]:
field_data01_39 = get_field_page_source(url ='http://www.ogb.state.ms.us/prodsearch.php') ## Done
#### field_data_40 is not crawled
# field_data_41_80 = get_field_page_source(url ='http://www.ogb.state.ms.us/prodsearch.php')  ##Done
# field_data_Bolton_Dixons_Bayou = get_field_page_source(url ='http://www.ogb.state.ms.us/prodsearch.php') 
# field_data_248_258 = get_field_page_source(url ='http://www.ogb.state.ms.us/prodsearch.php')
# field_data_258_268 =get_field_page_source(url ='http://www.ogb.state.ms.us/prodsearch.php')
# field_data_268_278 =get_field_page_source(url ='http://www.ogb.state.ms.us/prodsearch.php')
# field_data_278_288 =get_field_page_source(url ='http://www.ogb.state.ms.us/prodsearch.php')
# field_data_288_300 =get_field_page_source(url ='http://www.ogb.state.ms.us/prodsearch.php')

200
Abbott Field data is saved
Aberdeen Field data is saved
Aberdeen E. Field data is saved
Aberdeen S. Field data is saved
Addie Mae Field data is saved
Alcorn Field data is saved
ALFRED C. MOORE Field data is saved
Alloway Field data is saved
Amory S. Field data is saved
Anchorage Field data is saved
Anna Field data is saved
Ansley Field data is saved
Apollo E. Field data is saved
Apple Grove Field data is saved
Ariel N. Field data is saved
Arlington Field data is saved
Arm Dome Field data is saved
Armstrong Field data is saved
Armstrong S. Field data is saved
Arnot Field data is saved
Artonish Field data is saved
Artonish E Field data is saved
Ashwood Field data is saved
Ashwood E. Field data is saved
Ashwood S. Field data is saved
Athens Field data is saved
Auburn Field data is saved
Avera Field data is saved
Bacon Field data is saved
Bakers Creek Field data is saved
Bald Hill Field data is saved
Balls Branch Field data is saved
Barber Creek Field data is saved
Barber Creek E. Fiel

## Process and save the crawling data

In [14]:
def get_df_from_crawling_data(crawled_dict):
    
    ##container for saving the field dataframe
    all_fields= []
    ## generate df lists for fields
    for k in crawled_dict.keys():
        field = crawled_dict[k]
        each_field_stack = [] 
        ## store df_row in 'each_field_stack'
        for i in range(len( field)):
            row = field[i]
            df_row = pd.DataFrame(row)
            each_field_stack.append(df_row)
        ##concate one field rows together
        each_field = pd.concat(each_field_stack)
        all_fields.append(each_field)
    #concate field dataframe together
    all_fields_df = pd.concat(all_fields)
    return all_fields_df

In [15]:
## save the data
field_data01_03_preprocessed = get_df_from_crawling_data(field_data01_39)

In [16]:
field_data01_03_preprocessed.to_csv('field_data01_03_preprocessed')